In [2]:
#test join function
from os.path import join
re_aligned_read='  AATCCG   '
donor_genome='    TCGGCAT'
donor_genome=''.join([re_aligned_read[i] if donor_genome[i] == ' ' else donor_genome[i]\
                                        for i in range(len(donor_genome))])
donor_genome

'  AATCGGCAT'

In [3]:
''.join(['A','T','G',' ','G'])

'ATG G'

In [128]:
import numpy as np
def edit_distance_matrix(ref, donor):
    """
    Computes the edit distance matrix between the donor and reference

    This algorithm makes substitutions, insertions, and deletions all equal.
    Does that strike you as making biological sense? You might try changing the cost of
    deletions and insertions vs snps.
    :param ref: reference genome (as an ACTG string)
    :param donor: donor genome guess (as an ACTG string)
    :return: complete (len(ref) + 1) x (len(donor) + 1) matrix computing all changes
    """

    output_matrix = np.zeros((len(ref), len(donor)),dtype=int)
    # print len(ref), len(donor)
    # print output_matrix
    # This is a very fast and memory-efficient way to allocate a matrix
    penalty=1
    for i in range(len(ref)):
        output_matrix[i, 0] = penalty*i
    for j in range(len(donor)):
        output_matrix[0, j] = penalty*j
    for j in range(1, len(donor)):
        for i in range(1, len(ref)):  # Big opportunities for improvement right here.
            deletion = output_matrix[i - 1, j] + penalty #1. Make it bigger or smaller?
            insertion = output_matrix[i, j - 1] + penalty # 1
            identity = output_matrix[i - 1, j - 1]  if ref[i] == donor[j] else np.inf
            substitution = output_matrix[i - 1, j - 1] + 1 if ref[i] != donor[j] else np.inf
            output_matrix[i, j] = min(insertion, deletion, identity, substitution)
    backtrack_matrix=np.zeros
    return output_matrix

In [220]:
ref='TAAGCCCTAACCTTCCGACACTAAGGTGCCTCGAGACGGAAGATTCGAAATGAGGCAGTTGTTAAGCGTAGAGTCCCATGCCTTTGCAAGGTGGTGATCC'
donor='GGAAGATTCGAAATGAGGCAGTTGTTAAGCGTAGAGTCCCATGCCTTTGCAAGGTGGCGATCC'
#edit_matrix=edit_distance_matrix(ref,donor)
alignment(ref,donor,0)

0
38
1
TAAGCCCTAACCTTCCGACACTAAGGTGCCTCGAGACGGAAGATTCGAAATGAGGCAGTTGTTAAGCGTAGAGTCCCATGCCTTTGCAAGGTGGTGATCC
-------------------------------------GGAAGATTCGAAATGAGGCAGTTGTTAAGCGTAGAGTCCCATGCCTTTGCAAGGTGGCGATCC
[['SNP', 'T', 'C', 94]]


In [130]:
last_col=edit_matrix[:,-1]
last_row=edit_matrix[-1,:]
print(*edit_matrix)
# if last_row.min()<=last_col.min():
#     current_row = len(ref) - 1
#     current_column = last_row.argmin()
# else:
#     current_row = last_col.argmin()
#     current_column =len(ref) - 1
#print(*edit_matrix)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49
 50 51 52 53 54 55 56 57 58 59 60 61 62] [ 1  1  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61] [ 2  2  1  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22
 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60] [ 3  2  2  2  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21
 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46
 47 48 49 50 51 52 53 54 55 56 57 58 59] [ 4  3  3  3  2  2  3  4  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20
 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45
 46 47 48 49 50 51 52 53 54 55 56 57 58] [ 5  4  4  4  3  3  3  4  4  5  6  

In [131]:
e=edit_distance_matrix('$AATTCC','$AATCCGGG')
e

array([[0, 1, 2, 3, 4, 5, 6, 7, 8],
       [1, 0, 1, 2, 3, 4, 5, 6, 7],
       [2, 1, 0, 1, 2, 3, 4, 5, 6],
       [3, 2, 1, 0, 1, 2, 3, 4, 5],
       [4, 3, 2, 1, 1, 2, 3, 4, 5],
       [5, 4, 3, 2, 1, 1, 2, 3, 4],
       [6, 5, 4, 3, 2, 1, 2, 3, 4]])

In [134]:
def identify_changes(ref, donor, offset):
    """
    Performs a backtrace-based re-alignment of the donor to the reference and identifies
    SNPS, Insertions, and Deletions.
    Note that if you let either sequence get too large (more than a few thousand), you will
    run into memory issues.

    :param ref: reference sequence (ATCG string)
    :param donor: donor sequence (ATCG string)
    :param offset: The starting location in the genome.
    :return: SNPs, Inserstions, and Deletions
    """
    # print offset
    ref = '${}'.format(ref)  # add a dollar sign to the front
    donor = '${}'.format(donor)
    edit_matrix = edit_distance_matrix(ref=ref, donor=donor)
    #print edit_matrix
    #current_row = len(ref) - 1
    #current_column = len(donor) - 1
    last_col=edit_matrix[:,-1]
    last_row=edit_matrix[-1,:]
    if last_row.min()<=last_col.min():
        current_row = len(ref) - 1
        current_column = last_row.argmin()
    else:
        current_row = last_col.argmin()
        current_column =len(ref) - 1
    changes = []
    while current_row > 0 or current_column > 0:
        if current_row == 0:
            pvs_row = -np.inf
        else:
            pvs_row = current_row - 1

        if current_column == 0:
            pvs_column = -np.inf
        else:
            pvs_column = current_column - 1

        try:
            insertion_dist = edit_matrix[current_row, pvs_column]
        except IndexError:
            insertion_dist = np.inf

        try:
            deletion_dist = edit_matrix[pvs_row, current_column]
        except IndexError:
            deletion_dist = np.inf

        try:
            if ref[current_row] == donor[current_column]:
                identity_dist = edit_matrix[pvs_row, pvs_column]
            else:
                identity_dist = np.inf

            if ref[current_row] != donor[current_column]:
                substitution_dist = edit_matrix[pvs_row, pvs_column]
            else:
                substitution_dist = np.inf
        except (TypeError, IndexError) as e:
            identity_dist = np.inf
            substitution_dist = np.inf

        min_dist = min(insertion_dist, deletion_dist, identity_dist, substitution_dist)

        ref_index = current_row + offset - 1
        if min_dist == identity_dist:
            current_row = pvs_row
            current_column = pvs_column
        elif min_dist == substitution_dist:
            changes.append(['SNP', ref[current_row], donor[current_column], ref_index])
            current_row = pvs_row
            current_column = pvs_column
        elif min_dist == insertion_dist:
            try:
                if len(changes) > 0 and changes[-1][0] == 'INS' and changes[-1][-1] == ref_index + 1:
                    changes[-1][1] = donor[current_column] + changes[-1][1]
                else:
                    changes.append(['INS', donor[current_column], ref_index + 1])
            except IndexError:
                print(current_column)
            current_column = pvs_column
        elif min_dist == deletion_dist:
            if len(changes) > 0 and changes[-1][0] == 'DEL' and changes[-1][-1] == ref_index + 1:
                changes[-1] = ['DEL', ref[current_row] + changes[-1][1], ref_index]
            else:
                changes.append(['DEL', ref[current_row], ref_index])
            current_row = pvs_row
        else:
            raise ValueError
    changes = sorted(changes, key=lambda change: change[-1])
    print (str(changes))
    return changes

In [135]:
identify_changes(ref,donor,0)

64
[['INS', 'GGAAGA', 0], ['INS', 'TCG', 1], ['INS', 'AT', 3], ['INS', 'AGG', 4], ['INS', 'AGTTGTTAAG', 5], ['SNP', 'C', 'G', 6], ['INS', 'G', 9], ['INS', 'GT', 10], ['INS', 'CA', 12], ['SNP', 'T', 'G', 13], ['INS', 'TTT', 16], ['INS', 'C', 17], ['INS', 'AGGTGG', 18], ['INS', 'G', 19], ['INS', 'T', 20], ['INS', 'C', 21]]


[['INS', 'GGAAGA', 0],
 ['INS', 'TCG', 1],
 ['INS', 'AT', 3],
 ['INS', 'AGG', 4],
 ['INS', 'AGTTGTTAAG', 5],
 ['SNP', 'C', 'G', 6],
 ['INS', 'G', 9],
 ['INS', 'GT', 10],
 ['INS', 'CA', 12],
 ['SNP', 'T', 'G', 13],
 ['INS', 'TTT', 16],
 ['INS', 'C', 17],
 ['INS', 'AGGTGG', 18],
 ['INS', 'G', 19],
 ['INS', 'T', 20],
 ['INS', 'C', 21]]

In [25]:
def alignment(ref, donor, offset):

    # print offset
    ref = '${}'.format(ref)  # add a dollar sign to the front
    donor = '${}'.format(donor)
    edit_matrix = edit_distance_matrix(ref=ref, donor=donor)
    #print edit_matrix
    #current_row = len(ref) - 1
    #current_column = len(donor) - 1
    last_col=edit_matrix[:,-1]
    last_row=edit_matrix[-1,:]
    if last_row.min()<=last_col.min():
        current_row = len(ref) - 1
        current_column = last_row.argmin()
    else:
        current_row = last_col.argmin()
        current_column =len(donor) - 1
    aligned_ref=''
    aligned_donor=''
    while current_row > 0 or current_column > 0:
        if current_row == 0:
            pvs_row = -np.inf
        else:
            pvs_row = current_row - 1

        if current_column == 0:
            pvs_column = -np.inf
        else:
            pvs_column = current_column - 1

        try:
            insertion_dist = edit_matrix[current_row, pvs_column]
        except IndexError:
            insertion_dist = np.inf

        try:
            deletion_dist = edit_matrix[pvs_row, current_column]
        except IndexError:
            deletion_dist = np.inf

        try:
            if ref[current_row] == donor[current_column]:
                identity_dist = edit_matrix[pvs_row, pvs_column]
            else:
                identity_dist = np.inf

            if ref[current_row] != donor[current_column]:
                substitution_dist = edit_matrix[pvs_row, pvs_column]
            else:
                substitution_dist = np.inf
        except (TypeError, IndexError) as e:
            identity_dist = np.inf
            substitution_dist = np.inf

        min_dist = min(insertion_dist, deletion_dist, identity_dist, substitution_dist)

        ref_index = current_row + offset - 1
        if min_dist == identity_dist:
            
            aligned_ref=ref[current_row]+aligned_ref
            aligned_donor=donor[current_column]+aligned_donor
            current_row = pvs_row
            current_column = pvs_column
        elif min_dist == substitution_dist:
            aligned_ref=ref[current_row]+aligned_ref
            aligned_donor=donor[current_column]+aligned_donor
            current_row = pvs_row
            current_column = pvs_column
            
        elif min_dist == insertion_dist:
            aligned_ref='-'+aligned_ref
            aligned_donor=donor[current_column]+aligned_donor
            current_column = pvs_column
        elif min_dist == deletion_dist:
            aligned_ref=ref[current_row]+aligned_ref
            aligned_donor='-'+aligned_donor
            current_row = pvs_row
        else:
            raise ValueError
    print(aligned_ref)
    print(aligned_donor)


In [170]:
print(*alignment_matrix('$'+ref,'$'+donor))

[   0   -3   -6   -9  -12  -15  -18  -21  -24  -27  -30  -33  -36  -39  -42
  -45  -48  -51  -54  -57  -60  -63  -66  -69  -72  -75  -78  -81  -84  -87
  -90  -93  -96  -99 -102 -105 -108 -111 -114 -117 -120 -123 -126 -129 -132
 -135 -138 -141 -144 -147 -150 -153 -156 -159 -162 -165 -168 -171 -174 -177
 -180 -183 -186 -189 -192 -195 -198 -201 -204 -207 -210 -213 -216 -219 -222
 -225 -228 -231 -234 -237 -240 -243 -246 -249 -252 -255 -258 -261 -264 -267
 -270 -273 -276 -279 -282 -285 -288 -291] [-3  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0] [-6  0  2  1  0  0  0  0  0  1  1  0  0  0  0  0  0  0  1  0  1  0  0  1  1
  0  0  0  0  0  0  0  0  0  1  0  0  1  1  0  1  0  0  0  0  1  1  1  0  0
  1  0  0  0  1  0  0  0  0  0  0  1  1  0  0  0  0

In [211]:
# alignment matrix:
def alignment_matrix(ref,donor):
    import numpy as np
    output_matrix = np.zeros((len(ref), len(donor)),dtype=int)
    mismatch=-2
    match=2
    inse=-2
    dele=-2
    PM=np.zeros((len(ref)-1,len(donor)-1),dtype=int) # Pointer matrix (backtrack_matrix). right= 4, down= 3, Diagonal= 2
    Right=4
    Down=3
    Diagonal=2
    Start=1
    for i in range(len(ref)):
        output_matrix[i, 0] = dele*i
    for j in range(len(donor)):
        output_matrix[0, j] = inse*j
    for j in range(1, len(donor)):
        for i in range(1, len(ref)):  # Big opportunities for improvement right here.
            deletion = output_matrix[i - 1, j] + dele #1. Make it bigger or smaller?
            insertion = output_matrix[i, j - 1] + inse # 1
            identity = output_matrix[i - 1, j - 1] + match  if ref[i] == donor[j] else -np.inf
            substitution = output_matrix[i - 1, j - 1] + mismatch if ref[i] != donor[j] else -np.inf
            output_matrix[i, j] = max(insertion, deletion, identity, substitution,0)
            if output_matrix[i, j] ==identity or output_matrix[i, j] ==substitution:
                PM[i-1,j-1]=Diagonal
            elif output_matrix[i, j] ==deletion:
                PM[i-1,j-1]=Down
            elif output_matrix[i, j] ==insertion:#insertion
                PM[i-1,j-1]=Right
            else:#0, start position
                PM[i-1,j-1]=Start
    return output_matrix,PM                 

In [212]:
alignment_matrix('$TAACGCCCATC','$GCCCATCGGA')


(array([[  0,  -2,  -4,  -6,  -8, -10, -12, -14, -16, -18, -20],
        [ -2,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [ -4,   0,   0,   0,   0,   2,   0,   0,   0,   0,   2],
        [ -6,   0,   0,   0,   0,   2,   0,   0,   0,   0,   2],
        [ -8,   0,   2,   2,   2,   0,   0,   2,   0,   0,   0],
        [-10,   0,   0,   0,   0,   0,   0,   0,   4,   2,   0],
        [-12,   0,   2,   2,   2,   0,   0,   2,   2,   2,   0],
        [-14,   0,   2,   4,   4,   2,   0,   2,   0,   0,   0],
        [-16,   0,   2,   4,   6,   4,   2,   2,   0,   0,   0],
        [-18,   0,   0,   2,   4,   8,   6,   4,   2,   0,   2],
        [-20,   0,   0,   0,   2,   6,  10,   8,   6,   4,   2],
        [-22,   0,   2,   2,   2,   4,   8,  12,  10,   8,   6]]),
 array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 2, 4, 1, 1, 1, 2],
        [1, 1, 1, 1, 2, 2, 1, 1, 1, 2],
        [1, 2, 2, 2, 3, 2, 2, 4, 1, 3],
        [1, 3, 2, 2, 2, 1, 3, 2, 2, 4],
        [1, 2, 2, 

In [207]:
def alignment(ref, donor, offset):
    import numpy as np
    # print offset
    ref = '${}'.format(ref)  # add a dollar sign to the front
    donor = '${}'.format(donor)
    align_matrix,PM = alignment_matrix(ref=ref, donor=donor)
    match=1
    mismatch=-1
    insertion=-3
    deletion=-3
    #print edit_matrix
    #current_row = len(ref) - 1
    #current_column = len(donor) - 1
    last_col=align_matrix[:,-1]
    last_row=align_matrix[-1,:]
    aligned_ref=''
    aligned_donor=''
    if last_row.max()>=last_col.max():
        current_row = len(ref) - 1
        current_column = last_row.argmax()
        if last_row.argmax()<len(donor)-1:
            aligned_donor=donor[-(len(donor)-1-last_row.argmax()):]
            aligned_ref='-'*(len(donor)-1-last_row.argmax())
    else:
        current_row = last_col.argmax()
        current_column =len(donor) - 1
        aligned_donor='-'*(len(ref)-1-last_col.argmax())
        aligned_ref=ref[-(len(ref)-1-last_col.argmax()):]

    changes = []
    
    Right=2
    Down=1
    Diaganol=0
    while current_row > 0 or current_column > 0:#align_matrix[current_row,current_column]!=0: 
        #print (current_row,current_column)
        ref_index = current_row + offset - 1
        if current_row == 0:
            pvs_row = -np.inf
        else:
            pvs_row = current_row - 1
        if current_column == 0:
            pvs_column = -np.inf
        else:
            pvs_column = current_column - 1
        ###   
        
        
        
        
        
        try:
            if align_matrix[current_row, pvs_column]+ insertion == align_matrix[current_row, current_column]:#insertion
                aligned_ref='-'+aligned_ref
                aligned_donor=donor[current_column]+aligned_donor
                if len(changes) > 0 and changes[-1][0] == 'INS' and changes[-1][-1] == ref_index + 1:
                    changes[-1][1] = donor[current_column] + changes[-1][1]
                else:
                    changes.append(['INS', donor[current_column], ref_index + 1])
                current_column=pvs_column
                continue
        except IndexError:#pvs_column is -np.inf, which means the only way to go is go up. 
            aligned_ref=ref[current_row]+aligned_ref
            aligned_donor='-'+aligned_donor
            current_row=pvs_row
            continue
        try:
            if align_matrix[pvs_row, current_column]+ deletion == align_matrix[current_row, current_column]:#deletion
                aligned_ref=ref[current_row]+aligned_ref
                aligned_donor='-'+aligned_donor
                if len(changes) > 0 and changes[-1][0] == 'DEL' and changes[-1][-1] == ref_index + 1:
                    changes[-1] = ['DEL', ref[current_row] + changes[-1][1], ref_index]
                else:
                    changes.append(['DEL', ref[current_row], ref_index])
                current_row=pvs_row
                continue
        except IndexError:#pvs_row is -np.inf, which means the only way to go is to go left. 
            aligned_ref='-'+aligned_ref
            aligned_donor=donor[current_column]+aligned_donor
            current_column=pvs_column
            continue
            
        if align_matrix[pvs_row, pvs_column] + match ==align_matrix[current_row, current_column]:
            aligned_ref=ref[current_row]+aligned_ref
            aligned_donor=donor[current_column]+aligned_donor
            current_row=pvs_row
            current_column=pvs_column
            continue
            #print (current_row,current_column)
        if align_matrix[pvs_row, pvs_column] + mismatch ==align_matrix[current_row, current_column]:
            aligned_ref=ref[current_row]+aligned_ref
            aligned_donor=donor[current_column]+aligned_donor
            changes.append(['SNP', ref[current_row], donor[current_column], ref_index])
            current_row=pvs_row
            current_column=pvs_column
            continue
            #print (current_row,current_column)
        ####
        '''
        try:
            insertion_dist = align_matrix[current_row, pvs_column]
        except IndexError:
            insertion_dist = -np.inf

        try:
            deletion_dist = align_matrix[pvs_row, current_column]
        except IndexError:
            deletion_dist = -np.inf

        try:
            if ref[current_row] == donor[current_column]:
                identity_dist = align_matrix[pvs_row, pvs_column]
            else:
                identity_dist = -np.inf

            if ref[current_row] != donor[current_column]:
                substitution_dist = align_matrix[pvs_row, pvs_column]
            else:
                substitution_dist = -np.inf
        except (TypeError, IndexError) as e:
            identity_dist = -np.inf
            substitution_dist = -np.inf

        max_dist = max(insertion_dist, deletion_dist, identity_dist, substitution_dist)
        '''


        '''
        if min_dist == identity_dist:
            
            aligned_ref=ref[current_row]+aligned_ref
            aligned_donor=donor[current_column]+aligned_donor
            current_row = pvs_row
            current_column = pvs_column
        elif min_dist == substitution_dist:
            aligned_ref=ref[current_row]+aligned_ref
            aligned_donor=donor[current_column]+aligned_donor
            current_row = pvs_row
            current_column = pvs_column
            
        elif min_dist == insertion_dist:
            aligned_ref='-'+aligned_ref
            aligned_donor=donor[current_column]+aligned_donor
            current_column = pvs_column
        elif min_dist == deletion_dist:
            aligned_ref=ref[current_row]+aligned_ref
            aligned_donor='-'+aligned_donor
            current_row = pvs_row
        else:
            raise ValueError
        '''
    print(align_matrix[current_row,current_column])
    print(current_row)
    print(current_column)
    if current_row-1 > current_column-1:
        aligned_donor='-'*(current_row-current_column)+donor[1:current_column+1] + aligned_donor
        aligned_ref= ref[1:current_row+1]+aligned_ref
    print(aligned_ref)
    print(aligned_donor)
    print(changes)
    


In [218]:
ref='TAAGCCCTAACCTTCCGACACTAAGGTGCCTCGAGACGGAAGATTCGAAATGAGGCAGTTGTTAAGCGTAGAGTCCCATGCCTTTGCAAGGTGGTGATCC'
donor='TAAGCCCTAACCTTCCGACACTAAGGTGCCTCGAGGAAGATTCGAAATGAGGCAGTTGTTAAGCGTAGAGTCCCATGCCTTTGCAAGGTGGCGATCC'
alignment(ref,donor,0)
# alignment('TAACGCCCATC','GCCATCGGG',0)

0
0
0
TAAGCCCTAACCTTCCGACACTAAGGTGCCTCGAGACGGAAGATTCGAAATGAGGCAGTTGTTAAGCGTAGAGTCCCATGCCTTTGCAAGGTGGTGATCC
TAAGCCCTAACCTTCCGACACTAAGGTGCCTC--GA-GGAAGATTCGAAATGAGGCAGTTGTTAAGCGTAGAGTCCCATGCCTTTGCAAGGTGGCGATCC
[['SNP', 'T', 'C', 94], ['DEL', 'C', 36], ['DEL', 'GA', 32]]


In [217]:
alignment_matrix('$TAACGCCCATC','$GCCATC')
alignment('CGCCCATC','TAACGCCCATC',0)

0
1
4
---CGCCCATC
TAACGCCCATC
[]


In [57]:
String='ATCGGG'
String[-3:]

'GGG'

In [ ]:
# problem 33 find longest subsequence of two sequences
with open('rosalind_ba5c.txt','r') as f:
    s1=f.readline().strip()
    s2=f.readline().strip()

#build a alignment matrix
import numpy as np

AM=np.zeros((len(ref)+1,len(donor)+1)) #Alignment matrix. start from 0 position
PM=np.zeros((len(ref),len(donor))) # Pointer matrix (backtrack_matrix). right= 2, down= 1, diaganol= 0
Right=2
Down=1
Diaganol=0
for i in range(1,len(ref)+1):
    for j in range(1,len(donor)+1):
        ###
        AM=max(AM[i-1,j]-1,AM[i,j-1]-1,AM[i-1,j-1]+1 if ref[i-1]==donor[j-1] else AM[i-1,j-1]-1)
        ###
        '''
        if s1[i-1]==s2[j-1]:
            L=[AM[i-1,j],\
               AM[i,j-1],\
               AM[i-1,j-1]+ 1]
        else:
            L=[AM[i-1,j],\
               AM[i,j-1]]
        AM[i,j]=max(L)
        if L.index(AM[i,j])==0: # down pointer
            PM[i-1,j-1]=1
        elif L.index(AM[i,j])==1: # right pointer
            PM[i-1,j-1]=2
        elif L.index(AM[i,j])==2: # diagonal pointer
            PM[i-1,j-1]=0'''
        if AM==AM[i-1,j]-1:#deletion
            PM[i-1,j-1]=Down
        elif AM==AM[i,j-1]-1:#insertion
            PM[i-1,j-1]=Right
        else:#match or mismatch
            PM[i-1,j-1]=Diaganol
    
def backtrack(i,j,r,PM):
    Right=2
    Down=1
    Diaganol=0
    while i>=0 and j>=0:
        if PM[i,j]==Diaganol:
            r.append(Diaganol)
            i=i-1
            j=j-1
        elif PM[i,j]==Down:
            r.append(Down)
            i=i-1
        elif PM[i,j]==Right:
            r.append(Right)
            j=j-1
Route=[]
backtrack(len(s1)-1,len(s2)-1,Route, PM)
LCS=''
i=len(s2)-1
for e in Route:
    if e==Diaganol:# match
        LCS=s2[i]+LCS
        i=i-1
    elif e==Down: # down
        pass
    elif e==Right:
        i=i-1
LCS


In [227]:
def alignment(ref, donor, offset):
    import numpy as np
    # print offset
    ref = '${}'.format(ref)  # add a dollar sign to the front
    donor = '${}'.format(donor)
    align_matrix,PM = alignment_matrix(ref=ref, donor=donor)
    #print edit_matrix
    #current_row = len(ref) - 1
    #current_column = len(donor) - 1
    last_col=align_matrix[:,-1]
    last_row=align_matrix[-1,:]
    aligned_ref=''
    aligned_donor=''
    if last_row.max()>=last_col.max():
        current_row = len(ref) - 1
        current_column = last_row.argmax()
        if last_row.argmax()<len(donor)-1:
            aligned_donor=donor[-(len(donor)-1-last_row.argmax()):]
            aligned_ref='-'*(len(donor)-1-last_row.argmax())
    else:
        current_row = last_col.argmax()
        current_column =len(donor) - 1
        aligned_donor='-'*(len(ref)-1-last_col.argmax())
        aligned_ref=ref[-(len(ref)-1-last_col.argmax()):]
        
    changes = []
    Right=4
    Down=3
    Diagonal=2
    Start=1
    while current_row >= 1 and current_column >= 1:
        #print (current_row,current_column)
        ref_index = current_row + offset - 1 
        
        if PM[current_row-1,current_column-1]==Right:#insertion
            aligned_ref='-'+aligned_ref
            aligned_donor=donor[current_column]+aligned_donor
            if len(changes) > 0 and changes[-1][0] == 'INS' and changes[-1][-1] == ref_index + 1:
                changes[-1][1] = donor[current_column] + changes[-1][1]
            else:
                changes.append(['INS', donor[current_column], ref_index + 1])
            current_column=current_column-1
            
        elif PM[current_row-1,current_column-1]==Down:#deletion
            aligned_ref=ref[current_row]+aligned_ref
            aligned_donor='-'+aligned_donor
            if len(changes) > 0 and changes[-1][0] == 'DEL' and changes[-1][-1] == ref_index + 1:
                changes[-1] = ['DEL', ref[current_row] + changes[-1][1], ref_index]
            else:
                changes.append(['DEL', ref[current_row], ref_index])
            current_row=current_row-1
            
        elif PM[current_row-1,current_column-1]==Diagonal and ref[current_row]==donor[current_column]: #match
            aligned_ref=ref[current_row]+aligned_ref
            aligned_donor=donor[current_column]+aligned_donor
            current_row -=1
            current_column -=1
        elif PM[current_row-1,current_column-1]==Diagonal and ref[current_row]!=donor[current_column]: #mismatch
            aligned_ref=ref[current_row]+aligned_ref
            aligned_donor=donor[current_column]+aligned_donor
            changes.append(['SNP', ref[current_row], donor[current_column], ref_index])
            current_row -=1
            current_column -=1
        else: #start
            break

        ####
    print(align_matrix)
    print(current_row)
    print(current_column)
    if current_row-1 > current_column-1:
        aligned_donor='-'*(current_row-current_column)+donor[1:current_column+1] + aligned_donor
        aligned_ref= ref[1:current_row+1]+aligned_ref
    if current_row-1 < current_column-1:
        aligned_ref='-'*(current_column-current_row)+ref[1:current_row+1]+aligned_ref
        aligned_donor=donor[1:current_column+1]+aligned_donor
    print(aligned_ref)
    print(aligned_donor)
    print(changes)
    

In [231]:
#     alignment('PRETTY', 'PRTTEIN', offset=0)
#alignment(ref='ACACCC', donor='ATACCCGGG', offset=0)
#alignment(ref='ATACCCGGG', donor='ACACCC', offset=0)
#alignment(ref='ACACCC', donor='GGGATACCC', offset=0)
#alignment(ref='ACA', donor='AGA', offset=0)
#alignment(ref='ACA', donor='ACGTA', offset=0)
alignment(ref='TTACCGTGCAAGCG', donor='GCACCCAAGTTCG', offset=0)

[[  0  -2  -4  -6  -8 -10 -12 -14 -16 -18 -20 -22 -24 -26]
 [ -2   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [ -4   0   0   0   0   0   0   0   0   0   2   2   0   0]
 [ -6   0   0   2   0   0   0   2   2   0   0   0   0   0]
 [ -8   0   2   0   4   2   2   0   0   0   0   0   2   0]
 [-10   0   2   0   2   6   4   2   0   0   0   0   2   0]
 [-12   0   0   0   0   4   4   2   0   2   0   0   0   4]
 [-14   0   0   0   0   2   2   2   0   0   4   2   0   2]
 [-16   0   0   0   0   0   0   0   0   2   2   2   0   2]
 [-18   0   2   0   2   2   2   0   0   0   0   0   4   2]
 [-20   0   0   4   2   0   0   4   2   0   0   0   2   2]
 [-22   0   0   2   2   0   0   2   6   4   2   0   0   0]
 [-24   0   0   0   0   0   0   0   4   8   6   4   2   2]
 [-26   0   2   0   2   2   2   0   2   6   6   4   6   4]
 [-28   0   0   0   0   0   0   0   0   4   4   4   4   8]]
2
2
ACCGTGCAAG--CG
ACC---CAAGTTCG
[['INS', 'TT', 12], ['DEL', 'GTG', 5]]


In [ ]:
# SNV caller
#30x coverage  30*genome =100*read number   30/100=read number/genome=0.3
